# First approach

In [ ]:
# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import VotingClassifier, StackingClassifier

# Target encoding/decoding
from sklearn.base import BaseEstimator, TransformerMixin

# Metrics
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, auc, roc_curve, log_loss

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, plot_importance
from catboost import CatBoostClassifier

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Math and DataFrame
import pandas as pd
import numpy as np

# Warnings ignore
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
original = pd.read_csv('/content/steel_plates_faults_original_dataset.csv')
train.head()

In [ ]:
train.shape

In [ ]:
train = pd.concat([train, original], axis=0).drop_duplicates()
train.head()

In [ ]:
train.shape

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
train.head()

In [ ]:
target_variables = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps',  'Other_Faults']

In [ ]:
# If the sum of the axis is greater than 1 because in the same row more than 1 columns value is 1, we can sure that is not a multi class problem, it is multi label problem.
no_faults = train[train[target_variables].sum(axis=1) == 0][target_variables]
print(f'\nNumber of No Faults: {no_faults.shape[0]}\n')
no_faults

In [ ]:
no_faults.shape[0] / train.shape[0] * 100

In [ ]:
more_than_one_faults = train[train[target_variables].sum(axis=1) > 1][target_variables]
print(f'\nNumber of More than one faults: {more_than_one_faults.shape[0]}\n')
more_than_one_faults

In [ ]:
train = train[train[target_variables].sum(axis=1) <= 1]
train

In [ ]:
# Create a numpy array from our given targets.
targets_arr = train[target_variables].values.copy()
targets_arr

In [ ]:
no_faults_arr = 1 - targets_arr.sum(axis=1)[:, np.newaxis]
print(f'\nNumber of the No Faults: {no_faults_arr.sum()}\n')
no_faults_arr

In [ ]:
targets_concatenated = np.concatenate([targets_arr, no_faults_arr], axis=1)
targets_concatenated

In [ ]:
X = train.drop(target_variables, axis=1)
target = train[target_variables]
X.head()

In [ ]:
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)
test_sc = scaler.transform(test)
X_sc

### One-hot encoded to label-encoded

In [ ]:
label_encoded_targets = np.argmax(targets_concatenated, axis=-1)
label_encoded_targets

In [ ]:
unique_classes = np.unique(label_encoded_targets)
unique_classes

In [ ]:
class_weights = compute_class_weight(class_weight="balanced", classes=unique_classes, y=label_encoded_targets)
class_weights

In [ ]:
class_weights_param = {key: value for key, value in zip(unique_classes, class_weights)}
class_weights_param

### Optuna tuned parameters

In [ ]:
TUNE = False

In [ ]:
import optuna

# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the LGBMClassifier
    param = {
        "class_weight": class_weights_param,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 42,
        "num_class": 7,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03),
        "n_estimators": trial.suggest_int("n_estimators", 400, 600),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.005, 0.025),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.02, 0.06),
        "max_depth": trial.suggest_int("max_depth", 6, 14),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
    }

    # Create an instance of LGBMClassifier with the suggested parameters
    lgbm_classifier = LGBMClassifier(**param)

    # Fit the classifier on the training data
    lgbm_classifier.fit(X_train, y_train)

    # Evaluate the classifier on the test data
    score = lgbm_classifier.score(X_test, y_test)

    print(f'SCORE: {score}')

    return score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sc, label_encoded_targets, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

# If TUNE
if TUNE:
    # Run the optimization process
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=150)

    # Get the best parameters after optimization
    best_params = study.best_params

In [ ]:
# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the XGBClassifier
    param = {
        "objective": 'multi:softmax',
        "booster": 'gbtree',
        "random_state": 42,
        "num_class": 7,
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'gamma' : trial.suggest_float('gamma', 1e-9, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 0, 24),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 30),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
    }

    # Create an instance of LGBMClassifier with the suggested parameters
    xgb_classifier = XGBClassifier(**param)

    # Fit the classifier on the training data
    xgb_classifier.fit(X_train, y_train)

    # Evaluate the classifier on the test data
    score = xgb_classifier.score(X_test, y_test)
    y_pred_prob = xgb_classifier.predict_proba(X_test)
    lgbm_log_loss = log_loss(y_test, y_pred_prob)
    print(f'SCORE: {score}')
    print(f'Log Loss: {lgbm_log_loss}')

    return score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sc, label_encoded_targets, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

if TUNE:

    # Run the optimization process
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=150)

    # Get the best parameters after optimization
    best_params = study.best_params


In [ ]:
lgbm_params: dict = {
    "class_weight": class_weights_param, # Balanced class weight
    "objective": "multiclass",          # Objective function for the model
    "metric": "multi_logloss",          # Evaluation metric
    "verbosity": -1,                    # Verbosity level (-1 for silent)
    "boosting_type": "gbdt",            # Gradient boosting type
    "random_state": 42,       # Random state for reproducibility
    "num_class": 7,                     # Number of classes in the dataset
    'n_estimators': 752,
    'learning_rate': 0.005613916463106189,
    'max_depth': 6,
    'num_leaves': 252,
    'subsample': 0.6045538705062335,
    'colsample_bytree': 0.866501494211133,
    'colsample_bynode': 0.5443098861233086,
    'reg_alpha': 0.0024787490924597882,
    'reg_lambda': 3.5334079815178954,
    'min_split_gain': 0.05539710161546875,
}

In [ ]:
xgb_params: dict = {
    'class_weight': class_weights_param,
    'objective':'multi:softmax',
    'n_estimators': 829,
    'learning_rate': 0.010260565670497695,
    'gamma': 0.16282691057583543,
    'reg_alpha': 0.010492176264956674,
    'reg_lambda': 0.437536781187624,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6971737476610285,
    'colsample_bytree': 0.5115061295805807,
    'random_state': 345,
}

In [ ]:
cat_params: dict = {
    'class_weights': class_weights_param,
    'learning_rate': 0.13762007048684638,
    'depth': 5,
    'l2_leaf_reg': 5.285199432056192,
    'bagging_temperature': 0.6029582154263095,
    'random_seed': 42,
    'verbose': False,
    'iterations':1000,
}

In [ ]:
estimators = [
    ('XGB', XGBClassifier(**xgb_params)),
    ('LGBM', LGBMClassifier(**lgbm_params)),
    ('CAT', CatBoostClassifier(**cat_params))
]

In [ ]:
WEIGHT_TUNE = False

In [ ]:
# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the weighted ensemble

    obj_estimators = [
        ('XGB', XGBClassifier(**xgb_params)),
        ('LGBM', LGBMClassifier(**lgbm_params)),
        ('CAT', CatBoostClassifier(**cat_params))
    ]

    voting_classifier = VotingClassifier(
        estimators=obj_estimators,
        voting='soft',
        weights=[
            trial.suggest_float('XGB_Weight', 1.00, 9.00),
            trial.suggest_float('LGBM_Weight', 0.25, 5.00),
            trial.suggest_float('CAT_Weight', 0.25, 2.00),
        ],
    )

    voting_classifier.fit(X_train, y_train)

    predict_probs = voting_classifier.predict_proba(X_test)

    auc_score = roc_auc_score(y_test, predict_probs, multi_class='ovr')

    return auc_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sc, label_encoded_targets, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
weight_sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
weight_study = optuna.create_study(direction="maximize", sampler=weight_sampler)

if WEIGHT_TUNE:

    # Run the optimization process
    weight_study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=300)

    # Get the best parameters after optimization
    weight_best_params = weight_study.best_params

In [ ]:
voting_estimators = [
    ('XGB', XGBClassifier(**xgb_params)),
    ('LGBM', LGBMClassifier(**lgbm_params)),
    ('CAT', CatBoostClassifier(**cat_params))
]

In [ ]:
voting_classifier = VotingClassifier(
    estimators=voting_estimators,
    voting='soft',
    weights=[
            7.994761950304625,
            0.46883567511191715,
            0.4412142916220983
        ],
    )

voting_classifier.fit(X_sc, label_encoded_targets)

predicted_probs = voting_classifier.predict_proba(test_sc)[:, :-1] # Filtering out 8th's target (No Faults)

In [ ]:
predicted_probs.shape

In [ ]:
submission_df = pd.read_csv('/kaggle/input/playground-series-s4e3/sample_submission.csv')
submission_df

submission_df[target_variables] = predicted_probs
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)

# Second approach

In [ ]:
import sklearn
import numpy as np
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from prettytable import PrettyTable
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook.get_lock().locks = []
# !pip install sweetviz
# import sweetviz as sv
import concurrent.futures
from copy import deepcopy
from functools import partial
from itertools import combinations
import random
from random import randint, uniform
import gc
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,PowerTransformer, FunctionTransformer
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from itertools import combinations
from sklearn.impute import SimpleImputer
import xgboost as xg
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_squared_log_error, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, log_loss
from sklearn.cluster import KMeans
!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer
!pip install gap-stat
from gap_statistic.optimalK import OptimalK
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind
from scipy.stats import boxcox
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.base import BaseEstimator, TransformerMixin
!pip install optuna
import optuna
!pip install cmaes
import cmaes
import xgboost as xgb
!pip install catboost
!pip install lightgbm --install-option=--gpu --install-option="--boost-root=C:/local/boost_1_69_0" --install-option="--boost-librarydir=C:/local/boost_1_69_0/lib64-msvc-14.1"
import lightgbm as lgb
!pip install category_encoders
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder, CatBoostEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier,ExtraTreesClassifier, AdaBoostClassifier
!pip install -U imbalanced-learn
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from catboost import Pool
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option('display.max_columns',None)

In [ ]:
train=pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')
original=pd.read_csv("/kaggle/input/steel-plates-faults/SteelPlatesFaults.csv")

train.drop(columns=["id"],inplace=True)
test.drop(columns=["id"],inplace=True)

train_copy=train.copy()
test_copy=test.copy()
original_copy=original.copy()

print(original.shape)

device='cpu'

train=pd.concat([train,original],axis=0)
train.reset_index(inplace=True,drop=True)

target=['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']

original.head()

### Exploratory analysis

In [ ]:
cont_cols = test.columns
colors = ['blue', 'orange', 'green']

num_plots = len(cont_cols)
num_cols = 3
num_rows = -(-num_plots // num_cols)
fig, axes = plt.subplots(num_rows, num_cols, figsize=(21, 5 * num_rows))  # Adjust the figure size as needed

for i, feature in enumerate(cont_cols):
    row = i // num_cols
    col = i % num_cols

    ax = axes[row, col] if num_rows > 1 else axes[col]

    sns.histplot(train_copy[feature], kde=True, color=colors[0], label='Train', alpha=0.5, bins=30, ax=ax)
    sns.histplot(test_copy[feature], kde=True, color=colors[1], label='Test', alpha=0.5, bins=30, ax=ax)
    sns.histplot(original[feature], kde=True, color=colors[2], label='Original', alpha=0.5, bins=30, ax=ax)

    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')
    ax.legend()

if num_plots % num_cols != 0:
    for j in range(num_plots % num_cols, num_cols):
        axes[-1, j].axis('off')

plt.tight_layout()
plt.show()

### Feature engineering

In [ ]:
def OHE(train_df,test_df,cols,target):
    '''
    Function for one hot encoding, it first combined the data so that no category is missed and
    the category with least frequency can be dropped because of redunancy
    '''
    combined = pd.concat([train_df, test_df], axis=0)
    for col in cols:
        one_hot = pd.get_dummies(combined[col]).astype(int)
        counts = combined[col].value_counts()
        min_count_category = counts.idxmin()
        one_hot = one_hot.drop(min_count_category, axis=1)
        one_hot.columns=[str(f)+col for f in one_hot.columns]
        combined = pd.concat([combined, one_hot], axis="columns")
        combined = combined.loc[:, ~combined.columns.duplicated()]

    # split back to train and test dataframes
    train_ohe = combined[:len(train_df)]
    test_ohe = combined[len(train_df):]
    test_ohe.reset_index(inplace=True,drop=True)
    test_ohe.drop(columns=[target],inplace=True)
    return train_ohe, test_ohe

In [ ]:
cat_cols = [f for f in test.columns if test[f].nunique()/test.shape[0]*100<5 and test[f].nunique()>2 ]
test[cat_cols].nunique()

def nearest_val(target):
    return min(common, key=lambda x: abs(x - target))

global cat_cols_updated
cat_cols_updated=[]
for col in cat_cols:
    train[f"{col}_cat"]=train[col]
    test[f"{col}_cat"]=test[col]
    cat_cols_updated.append(f"{col}_cat")
    uncommon=list((set(test[col].unique())| set(train[col].unique()))-(set(test[col].unique())& set(train[col].unique())))
    if uncommon:
        common=list(set(test[col].unique())& set(train[col].unique()))
        train[f"{col}_cat"]=train[col].apply(nearest_val)
        test[f"{col}_cat"]=test[col].apply(nearest_val)

In [ ]:
def high_freq_ohe(train, test, extra_cols, target, n_limit=50):
    '''
    If you wish to apply one hot encoding on a feature with so many unique values, then this can be applied,
    where it takes a maximum of n categories and drops the rest of them treating as rare categories
    '''
    train_copy=train.copy()
    test_copy=test.copy()
    ohe_cols=[]
    for col in extra_cols:
        dict1=train_copy[col].value_counts().to_dict()
        ordered=dict(sorted(dict1.items(), key=lambda x: x[1], reverse=True))
        rare_keys=list([*ordered.keys()][n_limit:])
#         ext_keys=[f[0] for f in ordered.items() if f[1]<50]
        rare_key_map=dict(zip(rare_keys, np.full(len(rare_keys),9999)))

        train_copy[col]=train_copy[col].replace(rare_key_map)
        test_copy[col]=test_copy[col].replace(rare_key_map)
    train_copy, test_copy = OHE(train_copy, test_copy, extra_cols, target)
    drop_cols=[f for f in train_copy.columns if "9999" in f or train_copy[f].nunique()==1]
    train_copy=train_copy.drop(columns=drop_cols)
    test_copy=test_copy.drop(columns=drop_cols)

    return train_copy, test_copy

def cat_encoding(train, test, target):
    global overall_best_score
    global overall_best_col
    table = PrettyTable()
    table.field_names = ['Feature', 'Encoded Features', 'Log Loss Score']
    train_copy=train.copy()
    test_copy=test.copy()
    train_dum = train.copy()
    for feature in cat_cols_updated:
#         print(feature)
#         cat_labels = train_dum.groupby([feature])[target].mean().sort_values().index
#         cat_labels2 = {k: i for i, k in enumerate(cat_labels, 0)}
#         train_copy[feature + "_target"] = train[feature].map(cat_labels2)
#         test_copy[feature + "_target"] = test[feature].map(cat_labels2)

        dic = train[feature].value_counts().to_dict()
        train_copy[feature + "_count"] =train[feature].map(dic)
        test_copy[feature + "_count"] = test[feature].map(dic)

        dic2=train[feature].value_counts().to_dict()
#         list1=np.arange(len(dic2.values()),0,-1) # Higher rank for high count
        list1=np.arange(len(dic2.values())) # Higher rank for low count
        dic3=dict(zip(list(dic2.keys()),list1))

        train_copy[feature+"_count_label"]=train[feature].replace(dic3).astype(float)
        test_copy[feature+"_count_label"]=test[feature].replace(dic3).astype(float)

        temp_cols = [ feature + "_count", feature + "_count_label"]#feature + "_target",



        if train_copy[feature].nunique()<=5:
            train_copy[feature]=train_copy[feature].astype(str)+"_"+feature
            test_copy[feature]=test_copy[feature].astype(str)+"_"+feature
            train_copy, test_copy = OHE(train_copy, test_copy, [feature], target)

        else:
            train_copy,test_copy=high_freq_ohe(train_copy,test_copy,[feature], target, n_limit=5)
        train_copy=train_copy.drop(columns=[feature])
        test_copy=test_copy.drop(columns=[feature])


        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        auc_scores = []

        for f in temp_cols:
            X = train_copy[[f]].values
            y = train_copy[target].astype(int).values

            auc = []
            for train_idx, val_idx in kf.split(X, y):
                X_train, y_train = X[train_idx], y[train_idx]
                x_val, y_val = X[val_idx], y[val_idx]
                model =  HistGradientBoostingClassifier (max_iter=300, learning_rate=0.02, max_depth=6, random_state=42)
                model.fit(X_train, y_train)
                y_pred = model.predict_proba(x_val)[:,1]
                auc.append(roc_auc_score(y_val,  y_pred))
            auc_scores.append((f, np.mean(auc)))

        best_col, best_auc = sorted(auc_scores, key=lambda x: x[1], reverse=True)[0]

        corr = train_copy[temp_cols].corr(method='pearson')
        corr_with_best_col = corr[best_col]
        cols_to_drop = [f for f in temp_cols if corr_with_best_col[f] > 0.5 and f != best_col]
        final_selection = [f for f in temp_cols if f not in cols_to_drop]
        if cols_to_drop:
            train_copy = train_copy.drop(columns=cols_to_drop)
            test_copy = test_copy.drop(columns=cols_to_drop)

        table.add_row([feature, best_col, best_auc])
#         print(feature)

#     print(table)
    return train_copy, test_copy

In [ ]:
class Splitter:
    def __init__(self, test_size=0.2, kfold=True, n_splits=5):
        self.test_size = test_size
        self.kfold = kfold
        self.n_splits = n_splits

    def split_data(self, X, y, random_state_list):
        if self.kfold:
            for random_state in random_state_list:
                kf = KFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val

class Classifier:
    def __init__(self,n_estimators=100, device="cpu", random_state=0):
        self.n_estimators = n_estimators
        self.device = device
        self.random_state = random_state
        self.models = self._define_model()
        self.len_models = len(self.models)

    def _define_model(self):
        xgb_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.1,
            'max_depth': 4,
            'subsample': 0.8,
            'colsample_bytree': 0.1,
            'n_jobs': -1,
            'eval_metric': 'logloss',
            'objective': 'binary:logistic',
            'tree_method': 'hist',
            'verbosity': 0,
            'random_state': self.random_state,
#             'class_weight':class_weights_dict,
        }
        if self.device == 'gpu':
            xgb_params['tree_method'] = 'gpu_hist'
            xgb_params['predictor'] = 'gpu_predictor'

        xgb_params2=xgb_params.copy()
        xgb_params2['subsample']= 0.5
        xgb_params2['max_depth']=9
        xgb_params2['learning_rate']=0.045
        xgb_params2['colsample_bytree']=0.3

        xgb_params3=xgb_params.copy()
        xgb_params3['subsample']= 0.6
        xgb_params3['max_depth']=6
        xgb_params3['learning_rate']=0.02
        xgb_params3['colsample_bytree']=0.7

        xgb_params4=xgb_params.copy()
        xgb_params4['subsample']= 0.5943421542786502
        xgb_params4['max_depth']=6
        xgb_params4['learning_rate']=0.109
        xgb_params4['colsample_bytree']=0.5595039093313848
        lgb_params = {
            'n_estimators': self.n_estimators,
            'max_depth': 8,
            'learning_rate': 0.02,
            'subsample': 0.20,
            'colsample_bytree': 0.56,
            'reg_alpha': 0.25,
            'reg_lambda': 5e-08,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'device': self.device,
            'random_state': self.random_state,
            'verbose':-1,
#             'class_weight':class_weights_dict,
        }
        lgb_params2 = {
            'n_estimators': self.n_estimators,
            'max_depth': 5,
            'learning_rate': 0.015,
            'subsample': 0.50,
            'colsample_bytree': 0.1,
            'reg_alpha': 0.07608657669988828,
            'reg_lambda': 0.2255036530113883,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'device': self.device,
            'random_state': self.random_state,
        }
        lgb_params3=lgb_params.copy()
        lgb_params3['subsample']=0.9
        lgb_params3['reg_lambda']=0.3461495211744402
        lgb_params3['reg_alpha']=0.3095626288582237
        lgb_params3['max_depth']=8
        lgb_params3['learning_rate']=0.007
        lgb_params3['colsample_bytree']=0.5

        lgb_params4=lgb_params2.copy()
        lgb_params4['subsample']=0.3
        lgb_params4['reg_lambda']=0.49406951573373614
        lgb_params4['reg_alpha']=0.16269100796945424
        lgb_params4['max_depth']=9
        lgb_params4['learning_rate']=0.117
        lgb_params4['colsample_bytree']=0.3

        cb_params = {
            'iterations': self.n_estimators,
            'depth': 13,
            'learning_rate': 0.015,
            'l2_leaf_reg': 0.5,
            'random_strength': 0.1,
            'max_bin': 200,
            'od_wait': 65,
            'one_hot_max_size': 50,
            'grow_policy': 'Depthwise',
            'bootstrap_type': 'Bernoulli',
            'od_type': 'Iter',
            'eval_metric': 'AUC',
            'loss_function': 'Logloss',
            'task_type': self.device.upper(),
            'random_state': self.random_state,
        }
        cb_sym_params = cb_params.copy()
        cb_sym_params['grow_policy'] = 'SymmetricTree'
        cb_loss_params = cb_params.copy()
        cb_loss_params['grow_policy'] = 'Lossguide'

        cb_params2=  cb_params.copy()
        cb_params2['learning_rate']=0.01
        cb_params2['depth']=8

        cb_params3={
            'iterations': self.n_estimators,
            'random_strength': 0.5783342241486167,
            'one_hot_max_size': 10,
            'max_bin': 150,
            'learning_rate': 0.177,
            'l2_leaf_reg': 0.705662073971363,
            'grow_policy': 'SymmetricTree',
            'depth': 5,
            'max_bin': 200,
            'od_wait': 65,
            'bootstrap_type': 'Bayesian',
            'od_type': 'Iter',
            'eval_metric': 'AUC',
            'loss_function': 'Logloss',
            'task_type': self.device.upper(),
            'random_state': self.random_state,
        }
        cb_params4=  cb_params.copy()
        cb_params4['learning_rate']=0.01
        cb_params4['depth']=12
        dt_params= {'min_samples_split': 30, 'min_samples_leaf': 10, 'max_depth': 8, 'criterion': 'gini'}

        models = {
            'xgb': xgb.XGBClassifier(**xgb_params),
#            'xgb2': xgb.XGBClassifier(**xgb_params2),
#            'xgb3': xgb.XGBClassifier(**xgb_params3),
#            'xgb4': xgb.XGBClassifier(**xgb_params4),
#            'lgb': lgb.LGBMClassifier(**lgb_params),
#             'lgb2': lgb.LGBMClassifier(**lgb_params2),
#             'lgb3': lgb.LGBMClassifier(**lgb_params3),
#             'lgb4': lgb.LGBMClassifier(**lgb_params4),
            'cat': CatBoostClassifier(**cb_params),
#            'cat2': CatBoostClassifier(**cb_params2),
#             'cat3': CatBoostClassifier(**cb_params3),
#             'cat4': CatBoostClassifier(**cb_params4),
             "cat_sym": CatBoostClassifier(**cb_sym_params),
#             "cat_loss": CatBoostClassifier(**cb_loss_params)

        }
        return models


In [ ]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=5000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)

        auc_score = roc_auc_score(y_true, weighted_pred)
        log_loss_score=log_loss(y_true, weighted_pred)
        return auc_score#/log_loss_score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='maximize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)

    def weights(self):
        return self.weights


In [ ]:
def fit_model(X_train,X_test,y_train):
    kfold = True
    n_splits = 1 if not kfold else 5
    random_state = 2023
    random_state_list = [42] # used by split_data [71]
    n_estimators = 9999 # 9999
    early_stopping_rounds = 300
    verbose = False

    splitter = Splitter(kfold=kfold, n_splits=n_splits)

    # Initialize an array for storing test predictions
    test_predss = np.zeros(X_test.shape[0])
    y_train_pred=y_train.copy()

    ensemble_score = []
    weights = []
    trained_models = {'xgb':[], 'lgb':[]}


    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
        n = i % n_splits
        m = i // n_splits

        # Get a set of Regressor models
        classifier = Classifier(n_estimators, device, random_state)
        models = classifier.models

        # Initialize lists to store oof and test predictions for each base model
        oof_preds = []
        test_preds = []

        # Loop over each base model and fit it to the training data, evaluate on validation data, and store predictions
        for name, model in models.items():
            if ('cat' in name) or ("lgb" in name) or ("xgb" in name):
                if 'lgb' in name: #categorical_feature=cat_features
                    model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)]),#,categorical_feature=cat_features,)
                elif 'cat' in name:
                    model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)],#cat_features=cat_features,
                              early_stopping_rounds=early_stopping_rounds, verbose=verbose)
                else:
                    model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], early_stopping_rounds=early_stopping_rounds, verbose=0)
            else:
                model.fit(X_train_, y_train_)

            test_pred = model.predict_proba(X_test)[:, 1]
            y_val_pred = model.predict_proba(X_val)[:, 1]

            score = roc_auc_score(y_val, y_val_pred.reshape(-1, 1))
    #         score = accuracy_score(y_val, acc_cutoff_class(y_val, y_val_pred))

            print(f'{name} [FOLD-{n} SEED-{random_state_list[m]}] ROC AUC score: {score:.5f}')

            oof_preds.append(y_val_pred)
            test_preds.append(test_pred)

            if name in trained_models.keys():
                trained_models[f'{name}'].append(deepcopy(model))
        # Use Optuna to find the best ensemble weights
        optweights = OptunaWeights(random_state=random_state)
        y_val_pred = optweights.fit_predict(y_val.values, oof_preds)

        score = roc_auc_score(y_val, y_val_pred.reshape(-1, 1))
    #     score = accuracy_score(y_val, acc_cutoff_class(y_val, y_val_pred))
        print(f'Ensemble [FOLD-{n} SEED-{random_state_list[m]}] ------------------>  ROC AUC score {score:.5f}')
        ensemble_score.append(score)
        weights.append(optweights.weights)

        test_predss += optweights.predict(test_preds) / (n_splits * len(random_state_list))
        y_train_pred.loc[y_val.index]=np.array(y_val_pred)

        gc.collect()
    # Calculate the mean ROC AUC  score of the ensemble
    mean_score = np.mean(ensemble_score)
    std_score = np.std(ensemble_score)
    print(f'Ensemble ROC AUC score {mean_score:.5f} ± {std_score:.5f}')

    # Print the mean and standard deviation of the ensemble weights for each model
    print('--- Model Weights ---')
    mean_weights = np.mean(weights, axis=0)
    std_weights = np.std(weights, axis=0)
    for name, mean_weight, std_weight in zip(models.keys(), mean_weights, std_weights):
        print(f'{name}: {mean_weight:.5f} ± {std_weight:.5f}')
    print(f'Overall OOF Preds AUC SCORE {roc_auc_score(y_train,y_train_pred)}')

    print("__________________________________________________________________")
    return test_predss

In [ ]:
def post_processor(train, test):
    cols=test.columns.tolist()
    train_cop=train.copy()
    test_cop=test.copy()
    drop_cols=[]
    for i, feature in enumerate(cols):
        for j in range(i+1, len(cols)):
            if sum(abs(train_cop[feature]-train_cop[cols[j]]))==0:
                if cols[j] not in drop_cols:
                    drop_cols.append(cols[j])
    print(drop_cols)
    train_cop.drop(columns=drop_cols,inplace=True)
    test_cop.drop(columns=drop_cols,inplace=True)

    return train_cop, test_cop


In [ ]:
submission=pd.read_csv("/kaggle/input/playground-series-s4e3/sample_submission.csv")
submission.head()

In [ ]:
count=0
for col in target:
    train_temp=train[test.columns.tolist()+[col]]
    test_temp=test.copy()
    train_temp, test_temp= cat_encoding(train_temp, test_temp, col)

    final_features=test.columns.tolist()
    sc=StandardScaler()

    train_scaled=train_temp.copy()
    test_scaled=test_temp.copy()

    train_scaled[final_features]=sc.fit_transform(train[final_features])
    test_scaled[final_features]=sc.transform(test[final_features])

#     train_cop, test_cop=   post_processor(train_scaled, test_scaled)
    train_cop, test_cop= train_scaled, test_scaled
    X_train = train_cop.drop(columns=[col])
    y_train = train_cop[col]

    X_test = test_cop.copy()

    test_predss=fit_model(X_train,X_test,y_train)
    submission[col]=test_predss

    count+=1
    print(f'Column {col}, loop # {count}')

In [ ]:
submission.to_csv("submission_pure.csv",index=False)
submission.head()

In [ ]:
sub1=pd.read_csv("/kaggle/input/multiclass-feature-engineering-thoughts/submission.csv")
sub2=pd.read_csv("/kaggle/input/ps4e03-multi-class-lightgbm/submission.csv")

sub_list=[sub1, sub2,submission]
# weights=np.random.randint(1,4,len(sub_list))
weights=[1,1,1]

weighted_list = [item for sublist, weight in zip(sub_list, weights) for item in [sublist] * weight]

def ensemble_mean(sub_list,cols, mean="AM"):

    sub_out=sub_list[0].copy()
    if mean=="AM":
        for col in cols:
            sub_out[col]=sum(df[col] for df in sub_list)/len(sub_list)
    elif mean=="GM":
        for df in sub_list[1:]:

            for col in cols:
                sub_out[col]*=df[col]
        for col in cols:
            sub_out[col]=(sub_out[col])**(1/len(sub_list))
    elif mean=="HM":
        for col in cols:
            sub_out[col]=len(sub_list)/sum(1/df[col] for df in sub_list)
    sub_out[cols]=sub_out[cols].div(sub_out[cols].sum(axis=1), axis=0)

    return sub_out

sub_ensemble=ensemble_mean(weighted_list,target,mean="AM")
sub_ensemble.to_csv('submission.csv',index=False)
sub_ensemble.head()

# Third approach

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import OneHotEncoder, CatBoostEncoder, MEstimateEncoder
from sklearn.model_selection import StratifiedGroupKFold


from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression

from sklearn import set_config
import os
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from prettytable import PrettyTable

from sklearn.compose import make_column_transformer
from sklearn.base import clone
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from optuna.samplers import TPESampler
import optuna
pd.set_option("display.max_rows",100)

import tensorflow as tf
import random
os.environ['PYTHONHASHSEED'] = '51'
random.seed(89)